Sources of help:
https://stackoverflow.com/questions/12477823/calling-a-function-in-a-separate-file-in-python
https://stackoverflow.com/questions/20186344/ipynb-import-another-ipynb-file

https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.html
https://stackoverflow.com/questions/19482970/get-list-from-pandas-dataframe-column-headers
https://stackoverflow.com/questions/19379120/how-to-read-a-config-file-using-python
https://stackoverflow.com/questions/12943819/how-to-prettyprint-a-json-file
https://stackoverflow.com/questions/42370977/how-to-save-a-new-sheet-in-an-existing-excel-file-using-pandas

In [141]:
#data science stuff
import numpy as np
import pandas as pd
import pprint
import configparser
config = configparser.ConfigParser()
config.read('config.INI')

#from ipynb.fs.full.readMaps import *
#import ipynb.fs.full.readMaps
%run readMaps.ipynb
%run readMPLinks.ipynb
%run readTeams.ipynb
%run createLeaderboard.ipynb
%run osuApiRequests.ipynb
%run processMP.ipynb
%run sortScores.ipynb
%run sheetsAPI.ipynb

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

#requests
import requests
print("dab")

dab
google sheets authentication success
dab


In [9]:
#Get input
tourney_name = input("Name of tournament? ")

is_team_tourney = input("Is this a team tournament? (y/n) ")
is_team_tourney = True if is_team_tourney == 'y' else False

team_size = 0
if is_team_tourney == True:
    team_size = int(input("Team size? "))

#winning condition: score = 0, accuracy = 1, combo = 2, score v2 = 3
desired_scoring_type = input("What scoring? \
    (0)score, (1)acc, (2)combo, (3)scorev2" )
desired_scoring_type = str(desired_scoring_type)

#stage want to analyze
stage = input("What stage? ")

all_maps = readMaps('./Starter Sheets/Maps3.xlsx') #df of the maps and their mod in each stage
mp_ids = readMPLinks('./Starter Sheets/MPids.xlsx') #df of the mps in each stage


Name of tournament? OMF 2
Is this a team tournament? (y/n) y
Team size? 2
What scoring?     (0)score, (1)acc, (2)combo, (3)scorev23
What stage? Group Stage


In [142]:
#read teams sheet if is team tournament
teams = 0
teamsInfo = {}
if is_team_tourney == True:
    teams = readTeams('./Starter Sheets/Teams.xlsx')

    
#labels = list(all_maps)
#stages = labels[0::2] #stages in the tournament
#stage_idx = 0



#---Process the desired stage of the tournament---
maps_in_stage_df = all_maps.loc[:, stage:stage + " Mods"] #df of map_ids and mod
map_ids_in_stage = maps_in_stage_df.iloc[:, 0].tolist()
map_ids_in_stage = [map_id for map_id in map_ids_in_stage if map_id != '-1']
map_mods_in_stage = maps_in_stage_df.iloc[:, 1].tolist()
map_mods_in_stage = [mod for mod in map_mods_in_stage if mod != '-1']

mp_ids_in_stage = mp_ids[stage] #list of mp_ids in stage

mods_for_each_map = dict(zip(map_ids_in_stage, \
                            map_mods_in_stage)) #track the mod each map uses
map_counts_in_stage = {} #track how many times each map was picked
for map_id in map_ids_in_stage:
    map_counts_in_stage[map_id] = 0

#Create empty leaderboards for all the maps in a stage and append to 
#all_leaderboards
leaderboards_in_stage = {} #dict of map_ids each having a leaderboard
for map_id in map_ids_in_stage:
    if map_id == '0': #@TODO why are there still '0's?
        continue
    leaderboards_in_stage[map_id] = createLeaderboard(map_id, is_team_tourney)

#Parse each match in this stage. On each match we: append to appropriate
#leaderboard, update teaminfo, update map_count dict

for mp_id in mp_ids_in_stage:
    #@TODO: add team_info functionality
    processMP(mp_id, leaderboards_in_stage, map_counts_in_stage, 
              is_team_tourney, teams, desired_scoring_type, 
              mods_for_each_map)  

#---Sort the leaderboards by score---


#stage_idx += 2
pprint.pprint(leaderboards_in_stage)
print("-----------------------------------------------------------------")
    

#Construct the 'Overview', 'Best Plays', and 'Teams Overview' pages



     3rd err
     3rd err
     3rd err
     3rd err
     3rd err
     3rd err
     2nd err
     2nd err
     2nd err
     2nd err
     2nd err
     2nd err
     2nd err
     2nd err
     3rd err
     1st err: 3 45433386
     3rd err
     3rd err
     3rd err
     3rd err


AttributeError: 'function' object has no attribute 'pprint'

In [143]:
#create dict of map ids to map name
map_ids_to_names = {}
for map_id in map_counts_in_stage:
    map_name = get_beatmaps(map_id)[0]['title']
    map_ids_to_names[map_id] = map_name


In [31]:
#OUTDATED: create the local file containing all the leaderboards for stage
path = stage + '.xlsx'
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
for map_id in leaderboards_in_stage:
    leaderboards_in_stage[map_id]['scores'].to_excel(writer, sheet_name = map_id)
    
writer.save()
writer.close()

In [144]:
#create the google spreadsheet containing all the leaderboards for stage
body = {
    "properties": {
        "title": tourney_name + ' ' + stage + ' Leaderboards'
    },
    "sheets": []
}
for map_id in map_ids_to_names:
    body['sheets'].append({
        "properties":{
            "title": map_ids_to_names[map_id]
        }
    })
request = service.spreadsheets().create(body=body)
response = request.execute()
pprint(response)

leaderboard_spreadsheetId = response['spreadsheetId']

{'properties': {'autoRecalc': 'ON_CHANGE',
                'defaultFormat': {'backgroundColor': {'blue': 1,
                                                      'green': 1,
                                                      'red': 1},
                                  'padding': {'bottom': 2,
                                              'left': 3,
                                              'right': 3,
                                              'top': 2},
                                  'textFormat': {'bold': False,
                                                 'fontFamily': 'arial,sans,sans-serif',
                                                 'fontSize': 10,
                                                 'foregroundColor': {},
                                                 'italic': False,
                                                 'strikethrough': False,
                                                 'underline': False},
                                

In [145]:
#---Populate the new sheet with the leaderboards---
for map_id in leaderboards_in_stage:
    range_ = map_ids_to_names[map_id] + '!A1'
    
    #append the headers
    input_ = {
        "values": [
            leaderboards_in_stage[map_id]['scores'].columns.tolist()
        ]
    }
    request = service.spreadsheets().values().update(
        spreadsheetId = leaderboard_spreadsheetId, 
        range = range_,
        valueInputOption = 'USER_ENTERED',
        body = input_
    )
    response = request.execute()
    
    #append the leaderboard
    range_ = map_ids_to_names[map_id] + '!A2'
    scores = sortScores(leaderboards_in_stage[map_id]['scores'])
    input_ = {
        "values": scores.values.tolist()
    }
    request = service.spreadsheets().values().update(
        spreadsheetId = leaderboard_spreadsheetId, 
        range = range_,
        valueInputOption = 'USER_ENTERED',
        body = input_
    )
    response = request.execute()
    print(map_ids_to_names[map_id] + ': ')
    pprint(response)


ZETSUBOU: 
{'spreadsheetId': '1PO_ylf6JDKHwfQ7F64hhY7NpD0UsgvQR36t1ctZOLf8',
 'updatedCells': 252,
 'updatedColumns': 9,
 'updatedRange': 'ZETSUBOU!A2:I29',
 'updatedRows': 28}
Parallel Days: 
{'spreadsheetId': '1PO_ylf6JDKHwfQ7F64hhY7NpD0UsgvQR36t1ctZOLf8',
 'updatedCells': 549,
 'updatedColumns': 9,
 'updatedRange': "'Parallel Days'!A2:I62",
 'updatedRows': 61}
Kuusou Mesorogiwi: 
{'spreadsheetId': '1PO_ylf6JDKHwfQ7F64hhY7NpD0UsgvQR36t1ctZOLf8',
 'updatedCells': 1233,
 'updatedColumns': 9,
 'updatedRange': "'Kuusou Mesorogiwi'!A2:I138",
 'updatedRows': 137}
Absolute Demolition: 
{'spreadsheetId': '1PO_ylf6JDKHwfQ7F64hhY7NpD0UsgvQR36t1ctZOLf8',
 'updatedCells': 702,
 'updatedColumns': 9,
 'updatedRange': "'Absolute Demolition'!A2:I79",
 'updatedRows': 78}
Orange Diz: 
{'spreadsheetId': '1PO_ylf6JDKHwfQ7F64hhY7NpD0UsgvQR36t1ctZOLf8',
 'updatedCells': 675,
 'updatedColumns': 9,
 'updatedRange': "'Orange Diz'!A2:I76",
 'updatedRows': 75}
Senbonzakura (Short Ver.): 
{'spreadsheetId': '1PO